In [1]:
!pip install pymorphy3
!pip install emoji


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import emoji
from tqdm import tqdm  
import re

# Load synthetic texts
df = pd.read_csv('gen_texts_9916.csv', encoding='utf-8')
print(f"Loaded {len(df)} texts. Columns: {df.columns.tolist()}")


def clean_text(text):
    """Applies universal cleaning steps"""
    if not isinstance(text, str):
        return ""
    
    # Remove template artifacts
    text = re.sub(r'\[.*?\]|\<.*?\>', '', text)  
    
    # Normalize whitespace and quotes
    text = re.sub(r'\s+', ' ', text)
    text = text.replace('``', '"').replace("''", '"')
    
    return text.strip()

Loaded 9916 texts. Columns: ['Unnamed: 0', 'text', 'model', 'length']


In [31]:
df = df.applymap(lambda x: x.replace('#', '') if isinstance(x, str) else x)

C:\Users\KseniaLuschevaext\AppData\Local\Temp\ipykernel_16584\991729808.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace('#', '') if isinstance(x, str) else x)


In [3]:
def remove_emoji(text):
    if not isinstance(text, str):
        return text
    return ''.join(c for c in text if c not in emoji.EMOJI_DATA)

df = df.applymap(remove_emoji)

C:\Users\KseniaLuschevaext\AppData\Local\Temp\ipykernel_16584\5272915.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(remove_emoji)


In [4]:
# Apply to all texts
tqdm.pandas()
df['cleaned_text'] = df['text'].progress_apply(clean_text)
df.drop(columns=['Unnamed: 0'], inplace=True, axis=1)

100%|████████████████████████████████████████████████████████████████████████████| 9916/9916 [00:03<00:00, 3025.97it/s]


In [5]:
from razdel import sentenize  # Better than nltk for Russian
from pymorphy3 import MorphAnalyzer

morph = MorphAnalyzer()

def process_russian_text(text):
    """Performs sentence splitting and morphological normalization"""
    sentences = [s.text for s in sentenize(text)]
    processed_sentences = []
    
    for sent in sentences:
        # Tokenize and lemmatize (simple whitespace tokenizer for example)
        tokens = []
        for word in sent.split():
            parsed = morph.parse(word)[0]
            if parsed.tag.POS not in {'PREP', 'CONJ', 'PRCL'}:  # Filter some function words
                tokens.append(parsed.normal_form)
        
        processed_sentences.append(' '.join(tokens))
    
    return ' '.join(processed_sentences)

df['processed'] = df['cleaned_text'].progress_apply(process_russian_text)

# Check results
print("\nBefore processing:", df.iloc[0]['cleaned_text'])
print("\nAfter processing:", df.iloc[0]['processed'])

100%|██████████████████████████████████████████████████████████████████████████████| 9916/9916 [18:57<00:00,  8.72it/s]


Before processing: Сегодня мы обсудим загадочные и малоизученные области науки — каталитических нанотехнологий, которые играют ключевую роль в современной зелёной химией. Это не просто новый виток прогресса; это своего рода прощальный вздох старой эпохи химической индустрии, где природа остаётся важнейшим союзником человечества против загрязняющих производств. Катализаторы из мира наносистем словно тонкие нити между традиционными методами производства и новыми экологическими стандартами. Они позволяют снижать потребление ресурсов без утраты качества продукции. Но что скрывается за их изящными молекулярными структурами? Возможно ли сохранить эту гармонию природы и технологий навсегда? Давайте поразмышляем вместе об этих удивительных материалах, способных изменить ход истории экосистемы нашей планеты. Основная часть: В тени современных лабораторий скрыты крошечные герои борьбы за чистую планету - катализаторные комплексы размером меньше пылинки! Эти удивительные структуры способны управ

In [5]:
# Load natural texts
df2 = pd.read_csv('split_texts.csv', encoding='utf-8')
print(f"Loaded {len(df2)} texts. Columns: {df2.columns.tolist()}")

tqdm.pandas()
df2['cleaned_text'] = df2['text'].progress_apply(clean_text)

Loaded 44062 texts. Columns: ['Unnamed: 0', 'text', 'cleaned_text']


100%|██████████████████████████████████████████████████████████████████████████| 44062/44062 [00:17<00:00, 2537.79it/s]


In [6]:
df2['processed'] = df2['cleaned_text'].progress_apply(process_russian_text)

100%|██████████████████████████████████████████████████████████████████████████| 44062/44062 [1:41:43<00:00,  7.22it/s]


In [7]:
df2.head(10)

,Unnamed: 0,text,cleaned_text,processed
0,0,индивидуальный опыт прочтения звучит иногда та...,индивидуальный опыт прочтения звучит иногда та...,индивидуальный опыт прочтение звучать иногда т...
1,1,"хотя бы по хорошему, ясному русскому языку. и ...","хотя бы по хорошему, ясному русскому языку. и ...","хорошему, ясный русский языку. потом, кто чита..."
2,2,вообще достоевский сильно эксплуатирует своих ...,вообще достоевский сильно эксплуатирует своих ...,вообще достоевский сильно эксплуатировать свой...
3,3,талант писательский в первую очередь – талант ...,талант писательский в первую очередь – талант ...,талант писательский первый очередь – талант ча...
4,4,"не сломив этой силы, на преодолев достоевского...","не сломив этой силы, на преодолев достоевского...","сломить этот силы, преодолеть достоевский дост..."
5,5,"и в атеисте писатель видит не врага, а человек...","и в атеисте писатель видит не врага, а человек...","атеист писатель видеть врага, человека, достой..."
6,6,"вопрос о а.с. пушкине в романе в.в. набокова ""...","вопрос о а.с. пушкине в романе в.в. набокова ""...","вопрос а.с. пушкин роман в.в. набоков ""дар"". (..."
7,7,"роман в.в.набокова ""дар"" - это роман-исповедь,...","роман в.в.набокова ""дар"" - это роман-исповедь,...","роман в.в.набоков ""дар"" - роман-исповедь, ""исп..."
8,8,"но вопрос о человеке-поэте, как о смысле жизни...","но вопрос о человеке-поэте, как о смысле жизни...","вопрос человеке-поэте, смысл жизнь сам себя, с..."
9,9,"конкурс копирайта -к2 думаю, не окажусь очень ...","конкурс копирайта -к2 думаю, не окажусь очень ...","конкурс копирайт -к2 думаю, оказаться очень ор..."


In [9]:
df2.drop(columns='Unnamed: 0', axis=1, inplace=True)

In [10]:
df.to_csv('normalized_natural_text.csv')
df2.to_csv('normalized_synthetic_text.csv')

In [6]:
df['processed'].str.split().str.len().mean()
# Синтетические данные, ср. кол-во символов

np.float64(272.3143404598629)

In [16]:
df2['processed'].str.split().str.len().mean()
# Натуральные данные, ср. кол-во символов

np.float64(283.48508919250145)

In [35]:
df.to_csv('synthetic_text_new.csv')

In [34]:
df.iloc[10,0]

' **Корпусного анализа диалектических особенностей русской речи - забавная наука!**  Введение  Рассмотрение вопросов корпусного анализа диалектальных вариантов русского языка зачастую вызывает у специалистов улыбку и легкое недоумение среди широкого круга исследователей. Почему? Вообразите себе огромный архив народной словесности – тысячи текстов из самых разных уголков России! Эти тексты как россыпь драгоценностей, каждая со своим характерным колоритом и уникальной фонетикой. Но что происходит, когда мы пытаемся их проанализировать при помощи компьютерных программ? О божественная комедия начинается!  Основная часть  Итак, корпусной подход к изучению диалетализмов предполагает использование огромных массивов данных для выявления закономерностей развития языковой системы через время или пространство. Однако здесь кроется ирония судьбы — ведь компьютеры не всегда способны уловить тот неподражаемый дух народного говора. Например, программа может решить, что «чаво» означает нечто совсем др